In [1]:
import yfinance as yf
from apscheduler.schedulers.background import BackgroundScheduler
from datetime import datetime, timedelta
from pymongo import MongoClient

In [2]:
Vasanth= MongoClient("mongodb://<username>:Vasanth123@ac-1ijaqwn-shard-00-00.btiam0c.mongodb.net:27017,ac-1ijaqwn-shard-00-01.btiam0c.mongodb.net:27017,ac-1ijaqwn-shard-00-02.btiam0c.mongodb.net:27017/?ssl=true&replicaSet=atlas-5qjbna-shard-0&authSource=admin&retryWrites=true&w=majority")
Vasanth

MongoClient(host=['ac-1ijaqwn-shard-00-00.btiam0c.mongodb.net:27017', 'ac-1ijaqwn-shard-00-02.btiam0c.mongodb.net:27017', 'ac-1ijaqwn-shard-00-01.btiam0c.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, replicaset='atlas-5qjbna-shard-0', authsource='admin', retrywrites=True, w='majority', tls=True)

In [3]:
db=Vasanth['ICICIBank']

In [4]:
Collection=db['Data']

In [5]:
df = yf.download('ICICIBANK.NS', start="2019-07-1", end="2023-08-24")

[*********************100%%**********************]  1 of 1 completed


In [6]:
df=df.reset_index()
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-07-01,439.500000,442.000000,437.700012,439.000000,425.935608,6871979
1,2019-07-02,439.000000,440.000000,433.600006,436.750000,423.752563,7787303
2,2019-07-03,437.899994,440.000000,434.200012,435.950012,422.976379,10415896
3,2019-07-04,438.000000,441.399994,434.100006,436.200012,423.218994,11810688
4,2019-07-05,436.450012,439.899994,433.200012,436.350006,423.364471,12154330


In [7]:
df.shape

(1029, 7)

In [8]:
data=df.to_dict(orient="records")

In [9]:
data

[{'Date': Timestamp('2019-07-01 00:00:00'),
  'Open': 439.5,
  'High': 442.0,
  'Low': 437.70001220703125,
  'Close': 439.0,
  'Adj Close': 425.93560791015625,
  'Volume': 6871979},
 {'Date': Timestamp('2019-07-02 00:00:00'),
  'Open': 439.0,
  'High': 440.0,
  'Low': 433.6000061035156,
  'Close': 436.75,
  'Adj Close': 423.7525634765625,
  'Volume': 7787303},
 {'Date': Timestamp('2019-07-03 00:00:00'),
  'Open': 437.8999938964844,
  'High': 440.0,
  'Low': 434.20001220703125,
  'Close': 435.95001220703125,
  'Adj Close': 422.97637939453125,
  'Volume': 10415896},
 {'Date': Timestamp('2019-07-04 00:00:00'),
  'Open': 438.0,
  'High': 441.3999938964844,
  'Low': 434.1000061035156,
  'Close': 436.20001220703125,
  'Adj Close': 423.218994140625,
  'Volume': 11810688},
 {'Date': Timestamp('2019-07-05 00:00:00'),
  'Open': 436.45001220703125,
  'High': 439.8999938964844,
  'Low': 433.20001220703125,
  'Close': 436.3500061035156,
  'Adj Close': 423.3644714355469,
  'Volume': 12154330},
 {'Da

In [12]:
from apscheduler.schedulers.blocking import BlockingScheduler
# Define the stock ticker and time range
ticker = "ICICIBANK.NS"
start_time = "11:15:00"
end_time = "14:15:00"

# Define a function to fetch and store data
def fetch_store_data():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    
    if start_time <= current_time <= end_time:
        stock_data = yf.download(ticker, interval="15m", period="1d")
        
        if not stock_data.empty:
            data_dict = stock_data.iloc[-1].to_dict()
            data_dict["Date"] = data_dict.name
            collection.insert_one(data_dict)
            print(f"Data stored at {current_time}")

# Create an APS Scheduler instance
scheduler = BlockingScheduler()

# Schedule the job to run every 15 minutes
scheduler.add_job(fetch_store_data, 'interval', minutes=15)

# Start the scheduler
scheduler.start()
